| [Table of Contents](#table_of_contents) | [Data and model](#data_and_model) | [MLE](#mle) | [REMLE](#remle) | [References](#references) |

**Authors:** Andrej Gajdoš, Jozef Hanč, Martina Hančová <br> *[Faculty of Science](https://www.upjs.sk/en/faculty-of-science/?prefferedLang=EN), P. J. Šafárik University in Košice, Slovakia* <br> email: [andrej.gajdos@student.upjs.sk](mailto:andrej.gajdos@student.upjs.sk)
***
**<font size=6 color=brown> MLE, REMLE for cyber attacks</font>**  

<font size=4> R-based computational tools - </font> **<font size=4> fdslrm, nlme, MMEinR(mixed), sommer</font>**  
<a id=table_of_contents></a>
###  Table of Contents 
* [Data and model](#data_and_model) - data and model description of empirical data
* [MLE](#mle) - computation by standard packages *nlme*, *MMEinR(mixed)* and *fdslrm*
* [REMLE](#remle) - computation by standard package *nlme*, *MMEinR(mixed)*, *sommer* and *fdslrm*
* [References](#references)

**To get back to the table of contents, use <font color=brown>the Home key</font>.**

***
<a id=data_and_model></a>
# <font color=brown>Data and Model</font>

This FDSLRM application describes the real time series data set representing total weekly number of cyber attacks against a honeynet -- an unconventional tool which mimics real systems connected to Internet like business or school computers intranets to study methods, tools and goals of cyber attackers. 

Data, taken from from _Sokol, 2017_ were collected from November 2014 to May 2016 in CZ.NIC honeynet consisting of Kippo honeypots in medium-interaction mode. The number of time series observations is 
$n=72$. 

The suitable FDSLRM, after a preliminary logarithmic transformation of 
data $Z(t) = \log X(t)$, is Gaussian orthogonal:

$ 
\begin{array}{rl}
& Z(t) & \!  = \! &\beta_1+\beta_2\cos\left(\tfrac{2\pi t\cdot 3}{72}\right)+\beta_3\sin\left(\tfrac{2\pi t\cdot 3}{72}\right)+\beta_4\sin\left(\tfrac{2\pi t\cdot 4}{72}\right) + \\
&     &     & +Y_1\sin\left(\tfrac{2\pi\ t\cdot 6}{72}\right)+Y_2\sin\left(\tfrac{2\pi\cdot t\cdot 7}{72}\right)+w(t), \, t\in \mathbb{N}, 
\end{array}
$ 

where $\boldsymbol{\beta}=(\beta_1,\,\beta_2,\,\beta_3,\,\beta_4)' \in \mathbb{R}^4, \mathbf{Y} = (Y_1,Y_2)' \sim \mathcal{N}_2(\boldsymbol{0}, \mathrm{D}), w(t) \sim \mathcal{iid}\, \mathcal{N} (0, \sigma_0^2), \boldsymbol{\nu}= (\sigma_0^2, \sigma_1^2, \sigma_2^2) \in \mathbb{R}_{+}^3$.

We identified the given and most parsimonious structure of the FDSLRM using an iterative process of the model building and selection based on exploratory tools of *spectral analysis* and *residual diagnostics* (for details see our Jupyter notebook `cyberattacks.ipynb`).

In [1]:
# data - time series observation
x <- read.csv2('cyberattacks.csv', header = FALSE, sep = ",")
x <- as.numeric(as.vector(x[-1,]))
x <- log(x)
t <- 1:length(x)

In [2]:
# auxilliary functions to create design matrices F, V and the projection matrix M_F
makeF <- function(times, freqs) {

        n <- length(times)
        f <- length(freqs)
        c1 <- matrix(1, n)
        F <- matrix(c1, n, 1)

        for (i in 1:f) {
                F <- cbind(F, cos(2 * pi * freqs[i] * times))
                F <- cbind(F, sin(2 * pi * freqs[i] * times))
        }

        return(F)
}

makeV <- function(times, freqs) {

        V <- makeF(times, freqs)
        V <- V[, -1]

        return(V)
}

makeM_F <- function(F) {

        n <- nrow(F)
        c1 <- rep(1, n)
        I <- diag(c1)
        M_F <- I - F %*% solve((t(F) %*% F)) %*% t(F)

        return(M_F)
}

In [3]:
# model parameters
n <- 72 
k <- 4 
l <- 2 

# model - design matrices F, V
F <- makeF(t, c(3/72, 4/72))
F <- F[,-4]

V <- makeV(t, c(6/72, 7/72))
V <- V[,-c(1,3)]

***
<a id=mle></a>
# <font color=brown> MLE</font>

## nlme: Linear and Nonlinear Mixed Effects Models
* _Purpose_: R package to fit and compare Gaussian linear and nonlinear mixed-effects models 
* _Version_: 3.1-140, 2019
* _URL_: https://CRAN.R-project.org/package=nlme]
* _Key tool_: `lme` function for fitting LMM
* _Computational parameters_ of `lme`:
>   * <font size=2> `maxIter` - maximum number of iterations  (default: 50).
    * `msMaxIter` - maximum number of iterations for the optimization step (default: 50).
    * `tolerance` - tolerance for the convergence criterion (default: 1e-6).
    * `niterEM` - number of iterations for the EM algorithm used to refine the initial estimates of the random effects (default: 25).
    * `msMaxEval` - maximum number of evaluations of the objective function permitted for optimizer nlminb (default: 200).
    * `msTol` - tolerance for the convergence criterion on the first iteration when optim is used (default is 1e-7). 
    * `.relStep` - relative step for numerical derivatives calculations. Default is `.Machine$double.eps^(1/3)` while `.Machine$double.eps = 2.220446e-16`. 
    * `opt` - the optimizer to be used, either "nlminb" (the default) or "optim".
    * `optimMethod`  - the optimization method, a version of quasi-Newton method to be used with the optim optimizer (default:"BFGS", alternative: "L-BFGS-B")
    * `minAbsParApVar` - numeric value - minimum absolute parameter value in the approximate variance calculation (default: 0.05). <font>

In [4]:
library(nlme)

In [5]:
REMLE_nlme <- function(X, F, V, method) {
    g <- rep(1,length(X))
    d <- data.frame(g,F,V,X)
    colnames(d) <- c("g","f1","f2","f3","f4","v1","v2","X")
    
    result <- NULL
    
    if(method == "ML") {
        result <- lme(fixed=X~f2+f3+f4,random=list(g=pdDiag(~-1+v1+v2)),data=d,method="ML")
    } else {
        result <- lme(fixed=X~f2+f3+f4,random=list(g=pdDiag(~-1+v1+v2)),data=d,method="REML")
    }
    
    return(as.vector(c(result$sigma^2, diag(getVarCov(result)))))
}

In [6]:
# auxilliary function to calculate the norm of a vector
norm_vec <- function(x) sqrt(sum(x^2))

In [7]:
MLEnlme <- REMLE_nlme(x, F, V, "ML")
print(MLEnlme)
norm_vec(MLEnlme)

[1] 0.05595104 0.02392048 0.01394113


[1] 0.06242647

## fdslrm: Time series analysis and forecasting using LMM
    
* _Purpose_: R package for modeling and prediction of time series using linear mixed models.
* _Version_: 0.1.0, 2019
* _Depends_: kableExtra, IRdisplay, MASS, Matrix, car, nlme, stats, forecast, fpp2, matrixcalc, sommer, gnm, pracma, CVXR
* _Maintainer_: Andrej Gajdoš
* _Authors_: Andrej Gajdoš, Jozef Hanč, Martina Hančová
* _URL_: https://github.com/fdslrm/R-package
* _Installation_: Run jupyter notebook `00 installation fdslrm.ipynb` once before the first run of any R-based Jupyter notebook.

### fdslrm (nlme)
function `fitDiagFDSLRM` via parameter `"lme"` implements `lme` function from `nlme`
detailed explanat

In [8]:
library(fdslrm)
initialFDSLRM()


Attaching package: 'fdslrm'

The following objects are masked _by_ '.GlobalEnv':

    makeF, makeM_F, makeV



In [9]:
# fit particular FDSLRM employing ML estimation of variances (by 'lme' function)
MLEfitnlme <- fitDiagFDSLRM(x, t, freq_mean = c(3/72, 4/72), include_fixed_eff = c(1,1,0,1), freq_random = c(6/72, 7/72), 
                            include_random_eff = c(0,1,0,1), fit_function = "lme", var_estim_method = "ML")

In [10]:
print(as.vector(c(MLEfitnlme$error_variance, diag(MLEfitnlme$rand_eff_variance))))
norm_vec(as.vector(c(MLEfitnlme$error_variance, diag(MLEfitnlme$rand_eff_variance))))

[1] 0.05595104 0.02392048 0.01394113


[1] 0.06242647

## MMEinR
R version of Witkovský's MATLAB function `mixed` https://www.mathworks.com/matlabcentral/fileexchange/200
* _Purpose_: R function to estimate parameters of a linear mixed model (LMM) with a simple variance components structure
* _URL_: https://github.com/fdslrm/MMEinR
* _Computational parameters_ of `MMEinR (mixed)`:
>* <font size=2> tolerance for the convergence criterion (`epss = 1e-8`)
>* iterative method solving Henderson's mixed model equations <font>
>* function return the estimates of variance parameters in a different order: $(\tilde{\nu}_1,\ldots,\tilde{\nu}_l,\tilde{\nu}_0)'$ in comparison with other tools

In [11]:
# load function 'MMEinR'
devtools::source_url("https://github.com/fdslrm/MMEinR/blob/master/MMEinR.R?raw=TRUE")

SHA-1 hash of file is 36b3b4cd1a1f98c5fb36b541fc6b3956f6663aba
Warning message in mixed(y, X, Z, dim, s20, method):
"Estimated variance components are negative or zeros!"

In [12]:
MLEmixed <- mixed(x, F, V, c(1,1), c(1,1,1), 1)
print(MLEmixed$s2)
print(norm_vec(MLEmixed$s2))

[1] 0.02392048 0.01394113 0.05595104
[1] 0.06242647


### fdslrm (MMEinR)
function `fitDiagFDSLRM` via parameter `"mixed"` implements R version of MATLAB `mixed` function

In [13]:
# fit particular FDSLRM employing ML estimation of variances (by 'mixed' function)
MLEfitmixed <- fitDiagFDSLRM(x, t, freq_mean = c(3/72, 4/72), include_fixed_eff = c(1,1,0,1), freq_random = c(6/72, 7/72), 
                             include_random_eff = c(0,1,0,1), fit_function = "mixed", var_estim_method = "ML")

In [14]:
print(as.vector(c(MLEfitmixed$error_variance, MLEfitmixed$rand_eff_variance)))
norm_vec(as.vector(c(MLEfitmixed$error_variance, MLEfitmixed$rand_eff_variance)))

[1] 0.05595104 0.02392048 0.01394113


[1] 0.06242647

***
<a id=remle></a>
# <font color=brown> REMLE</font>

## nlme

In [15]:
REMLEnlme <- REMLE_nlme(x, F, V, "REML")
print(REMLEnlme)
norm_vec(REMLEnlme)

[1] 0.05934201 0.02382629 0.01384694


[1] 0.06542862

### fdslrm (nlme)

In [16]:
# fit particular FDSLRM employing REML estimation of variances (by 'lme' function)
REMLEfitnlme <- fitDiagFDSLRM(x, t, freq_mean = c(3/72, 4/72), include_fixed_eff = c(1,1,0,1), freq_random = c(6/72, 7/72), 
                              include_random_eff = c(0,1,0,1), fit_function = "lme", var_estim_method = "REML")

In [17]:
print(as.vector(c(REMLEfitnlme$error_variance, diag(REMLEfitnlme$rand_eff_variance))))
norm_vec(as.vector(c(REMLEfitnlme$error_variance, diag(REMLEfitnlme$rand_eff_variance))))

[1] 0.05934201 0.02382629 0.01384694


[1] 0.06542862

## MMEinR

In [18]:
MLEmixed <- mixed(x, F, V, c(1,1), c(1,1,1), 2)
print(MLEmixed$s2)
print(norm_vec(MLEmixed$s2))

[1] 0.02382629 0.01384694 0.05934201
[1] 0.06542862


###  fdslrm (MMEinR)

In [19]:
# fit particular FDSLRM employing REML estimation of variances (by 'MMEinR (mixed)' function)
REMLEfitmixed <- fitDiagFDSLRM(x, t, freq_mean = c(3/72, 4/72), include_fixed_eff = c(1,1,0,1), freq_random = c(6/72, 7/72), 
                               include_random_eff = c(0,1,0,1), fit_function = "mixed", var_estim_method = "REML")

In [20]:
print(as.vector(c(REMLEfitmixed$error_variance, REMLEfitmixed$rand_eff_variance)))
norm_vec(as.vector(c(REMLEfitmixed$error_variance, REMLEfitmixed$rand_eff_variance)))

[1] 0.05934201 0.02382629 0.01384694


[1] 0.06542862

## sommer: Solving Mixed Model Equations in R
* _Purpose_: R package - structural multivariate-univariate linear mixed model solver for multiple random effects and estimation of unknown variance-covariance structures (i.e. heterogeneous and unstructured variance models).   
<font size=2> Designed for genomic prediction and genome wide association studies (GWAS), particularly focused in the $p$ > $n$ problem (more coefficients than observations) to include multiple known relationship matrices and estimate complex unknown covariance structures. Spatial models can be fitted using the two-dimensional spline functionality in sommer. </font>

* _Version_: 3.9.3, 2019
* _URL_: https://CRAN.R-project.org/package=sommer
* _Key tool_: `mmer` function for fitting
* _computational parameters_ of `mmer`:
>   * <font size=2> `iters` - maximum number of iterations (default: 20).
    * `tolpar` - tolerance for the convergence criterion for the change in loglikelihood (default: 1e-03)
    * `tolparinv` - tolerance parameter for matrix inverse used when singularities are encountered (default:1e-06). 
    * `method` - this refers to the method or algorithm to be used for estimating variance components (default: NR, direct-inversion Newton-Raphson method, alternative: AI,  Average Information) 
    * function return the estimates of variance parameters in a different order: $(\tilde{\nu}_1,\ldots,\tilde{\nu}_l,\tilde{\nu}_0)'$ in comparison with other tools<font>

In [21]:
library(sommer)

In [22]:
REMLE_sommer <- function(X, F, V) {
    f1 <- F[,1]
    f2 <- F[,2]
    f3 <- F[,3]
    f4 <- F[,4]
    v1 <- V[,1]
    v2 <- V[,2]
    
    DT <- data.frame(X, f1, f2, f3, f4, v1, v2)

    suppressWarnings(result_new <- mmer(fixed = X~f2+f3+f4, random = ~ vs(ds(v1),1)+vs(ds(v2),1), data = DT, verbose = FALSE))
    
    output <- as.vector(unlist(result_new$sigma))
    
    return(output)

}

In [23]:
REMLEsommer <- REMLE_sommer(x, F, V)
print(REMLEsommer)
print(norm_vec(REMLEsommer))

[1] 0.02382450 0.01384666 0.05934209
[1] 0.06542798


### fdslrm (sommer)
function `fitDiagFDSLRM` via parameter `"mmer"` implements `mmer` function from `sommer`

In [24]:
# fit particular FDSLRM employing REML estimation of variances (by 'mmer' function)
REMLEfitsommer <- suppressWarnings(fitDiagFDSLRM(x, t, freq_mean = c(3/72, 4/72), include_fixed_eff = c(1,1,0,1), 
                                                 freq_random = c(6/72, 7/72), include_random_eff = c(0,1,0,1), 
                                                 fit_function = "mmer", var_estim_method = "REML"))

fixed-effect model matrix is rank deficient so dropping 1 columns / coefficients


In [25]:
print(as.vector(c(REMLEfitsommer$error_variance, REMLEfitsommer$rand_eff_variance)))
print(norm_vec(c(REMLEfitsommer$error_variance, REMLEfitsommer$rand_eff_variance)))

[1] 0.05934209 0.02382450 0.01384666
[1] 0.06542798


***
<a id=references></a>
# <font color=brown> References </font>
This notebook belongs to suplementary materials of the paper submitted to Statistical Papers and available at <https://arxiv.org/abs/1905.07771>.

* Hančová, M., Vozáriková, G., Gajdoš, A., Hanč, J. (2019). [Estimating variance components in time series
	linear regression models using empirical BLUPs and convex optimization](https://arxiv.org/abs/1905.07771), https://arxiv.org/, 2019.

### Abstract of the paper

We propose a two-stage estimation method of variance components in time series models known as FDSLRMs, whose observations can be described by a linear mixed model (LMM). We based estimating variances, fundamental quantities in a time series forecasting approach called kriging, on the empirical (plug-in) best linear unbiased predictions of unobservable random components in FDSLRM. 

The method, providing invariant non-negative quadratic estimators, can be used for any absolutely continuous probability distribution of time series data. As a result of applying the convex optimization and the LMM methodology, we resolved two problems $-$ theoretical existence and equivalence between least squares estimators, non-negative (M)DOOLSE, and maximum likelihood estimators, (RE)MLE, as possible starting points of our method and a 
practical lack of computational implementation for FDSLRM. As for computing (RE)MLE in the case of $ n $ observed time series values, we also discovered a new algorithm of order $\mathcal{O}(n)$, which at the default precision is $10^7$ times more accurate and $n^2$ times faster than the best current Python(or R)-based computational packages, namely CVXPY, CVXR, nlme, sommer and mixed. 

We illustrate our results on three real data sets $-$ electricity consumption, tourism and cyber security $-$ which are easily available, reproducible, sharable and modifiable in the form of interactive Jupyter notebooks.

* Sokol P., Gajdoš, A. (2017). [Prediction of Attacks Against Honeynet Based on Time Series Modeling](https://www.springer.com/gp/book/9783319676203). Silhavy, R., Silhavy, P., & Prokopova, Z. (Eds.). (2017). _Applied Computational Intelligence and Mathematical Methods (Vol. 662)_. Cham: Springer International Publishing, pp. 360-371

| [Table of Contents](#table_of_contents) | [Data and model](#data_and_model) | [MLE](#mle) | [REMLE](#remle) | [References](#references) |